<span style="color:red">Abgegeben von (Name, Vorname):</span> 
**Elsherif, Mohamed**

In [1]:
import nltk

Immer griffbereit:
- Website: https://www.nltk.org/
- Buch: https://www.nltk.org/book/
- Module: https://www.nltk.org/py-modindex.html
- Beispiele: http://www.nltk.org/howto/

<br>
<font size="6"><strong>7. Sitzung: POS-Tagging und N-Gram-Modelle</strong></font>
<br>

Das POS-Tagging ist wie die Tokenisierung ein wichtiger Vorverarbeitungsschritt bei der Analyse von Sätzen. In diesem Notebook werden wir einfache POS-Tagger mittels N-Gram-Modelle implementieren.

# Was ist POS-Tagging?

Unter POS-Tagging versteht man die **Wortartenklassifizierung** von Wordtoken anhand sogenannter **POS-Tags**. Zum Beispiel wird im ersten Satz des Brown Corpus ein Wordtoken *place* mit dem POS-Tag `NN` versehen, um anzuzeigen, dass es sich hierbei um ein Nomen handelt. 

In [2]:
from nltk.corpus import brown

brown.tagged_sents()[0][23]

('place', 'NN')

Das POS-Tagging hilft, von der konkreten Wortform eines Tokens zu abstrahieren und (statistische) Zusammenhänge zwischen Worttoken leichter zu erkennen. Damit lässt sich z.B. leichter der grobe Bauplan eines Satzes angeben, ohne alle möglichen Wortformen aufzählen zu müssen. Wir können dann Beschreibungen benutzen wie: 

- Ein Nomen steht oft direkt hinter einem Artikel.
- Ein Artikel steht nie direkt hinter einem Artikel (zumindest im Englischen).
- Ein Adjektiv steht oft zwischen einem Artikel und einem Nomen. 
- usw.

POS-Tags könnnen auch bei Suchanfragen verwendet werden, etwa um [Konkordanzen](#Appendix:-Konkordanzen) zu erstellen.

# Wortarten und Tagsets

Ein POS-Tag zeigt die [**Wortart**](https://de.wikipedia.org/wiki/Wortart) (engl.: part of speech) eines Wortokens an. Die Wortart wird abhängig von der Morphologie (d.h. Wortform), der Syntax (d.h. der Umgebung des Worttokens im Satz) und der Semantik (d.h. Wortbedeutung) bestimmt und soll wichtige Eigenschaften dieser Aspekte zusammenfassen.

**Beispiel:** Nomen (Substantiv, engl.: noun)
- *Morphologie*: Ein Nomen hat bestimmte Merkmale (z.B. Kasus, Numerus, Genus), ggf. erkennbar an bestimmten Derivationsaffixen (*open*-***ness***) und Flexionsaffixen (*house*-***s***).
- *Syntax*: Ein Nomen steht hinter einem Artikel oder Adjektiv und bildet mit diesen eine [Konstituente](https://de.wikipedia.org/wiki/Konstituente) oder [Phrase](https://de.wikipedia.org/wiki/Phrase_(Linguistik)).
- *Semantik*: Ein Nomen bezeichnet (im prototypischen Fall) eine Sache.

Je nach Auswahl und Granularität der morphologischen, syntaktischen und semantischen Eigenschaften können unterschiedliche **Tagsets** definiert werden. Für das Englische gibt es zum Beispiel drei sehr weit verbreitete Tagsets:

- [Tagset des Brown Corpus](https://varieng.helsinki.fi/CoRD/corpora/BROWN/tags.html) (87 Tags)
- [Tagset der Penn Treebank (PTB)](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) (36 POS-Tags + 13 sonstige Tags)
- [Tagset des British National Corpus (BNC)](http://www.natcorp.ox.ac.uk/docs/c5spec.html) (62 Tags)

Für das Deutsche hat sich das [Stuttgart-Tübingen Tagset (STTS)](https://homepage.ruhr-uni-bochum.de/Stephen.Berman/Korpuslinguistik/Tagsets-STTS.html) durchgesetzt.

Der Umfang eines Tagsets hängt nicht zuletzt davon ab, wie reich die Morphologie einer Sprache ist. Die Tagsets für morphologisch reichere Sprachen sind oft (aber nicht immer) umfangreicher als Tagsets für morphologisch einfachere Sprachen wie das Englische. Z.B. gibt es ein [Tagset für das Tschechische](https://www.sketchengine.eu/tagset-reference-for-czech/), das 4288 POS-Tags enthält.    

NLTK stellt bei den getaggten Corpora neben dem ursprünglichen Tagset auch ein sogenanntes **Universelles Tagset** zur Verfügung. Dieses enthält im Wesentlichen die Hauptwortarten, die auch im Schulunterricht behandelt werden (und aus der griechischen/lateinischen Grammatikschreibung stammen). 

NLTK dokumentiert die genutzen POS-Tags aus dem Universellen Tagset folgendermaßen:

| **Tag** | **Meaning**         | **English Examples**                     |
| :------- | :------------------- | :---------------------------------------- |
| `ADJ`   | adjective           | *new, good, high, special, big, local*   |
| `ADP`   | adposition          | *on, of, at, with, by, into, under*      |
| `ADV`   | adverb              | *really, already, still, early, now*     |
| `CONJ`  | conjunction         | *and, or, but, if, while, although*      |
| `DET`   | determiner, article | *the, a, some, most, every, no, which*   |
| `NOUN`  | noun                | *year, home, costs, time, Africa*        |
| `NUM`   | numeral             | *twenty-four, fourth, 1991, 14:24*       |
| `PRT`   | particle            | *at, on, out, over, per, that, up, with*  |
| `PRON`  | pronoun             | *he, their, her, its, my, I, us*         |
| `VERB`  | verb                | *is, say, told, given, playing, would*   |
| `.`     | punctuation marks   | *. , ; \!*                               |
| `X`     | other               | *ersatz, esprit, dunno, gr8, univeristy* |

Das vollständige Universelle Tagset enthält außerdem `AUX` ("auxiliary verb"), `INTJ` ("interjection"), `PROPN` ("proper noun"), `SCONJ` ("subordinating conjunction") und `SYM` ("symbol").

Man beachte: Mehrworteinheiten wie *Windows 7* werden nicht als ganzes getagt, sondern nur jeweils die darin enthaltenen Wörter. Das Ergebnis wäre hier `PROPN` und `NUM`. 

Das Universelle Tagset kann mittels der Option `tagset='universal'` aufgerufen werden:

In [3]:
nltk.download('universal_tagset')
print("Brown Tagset:\n{}\n".format(brown.tagged_sents()[0]))
print("Universal Tagset:\n{}".format(brown.tagged_sents(tagset='universal')[0]))

Brown Tagset:
[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("Atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'NN'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'CS'), ('any', 'DTI'), ('irregularities', 'NNS'), ('took', 'VBD'), ('place', 'NN'), ('.', '.')]

Universal Tagset:
[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')]


[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/dr.elsherif/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


Das Universelle Tagset wurde im Rahmen der [**Universal-Dependencies-Initiative**](https://universaldependencies.org/) definiert und soll die Tagsets sprachübergreifend vereinheitlichen (weitere Informationen gibt es [hier](https://universaldependencies.org/u/pos/)).

## <span style="color:red">Aufgaben I: Ambiguität beim POS-Tagging</span>

Für das POS-Tagging reicht es oft nicht aus, nur die Wortform eines Tokens zu betrachten, denn oft kann dieselbe Wortformen unterschiedlichen POS-Tags zugeordnet werden. Wortformen sind also hinsichtlich der POS-Tags mehrdeutig/ambig.

<span style="color:red">A1:</span> Bestimmen Sie den Grad der POS-Tag-Ambiguität der Wortformen im Brown Corpus bezogen auf das Universelle Tagset anhand der folgenden Kennzahlen:
- durchschnittliche Anzahl der POS-Tags einer Wortform
- die 10 Wortformen mit der höchsten Anzahl unterschiedlicher POS-Tags
- der %-Anteil der ambigen Wortformen bei den Wortformen
- durchschnittliche Anzahl der möglichen POS-Tags eines Worttokens **aufgrund seiner Wortform**
- der %-Anteil der Worttoken mit ambiger Wortform

Hinweis: Hier können Sie z.B. [FreqDist](https://www.nltk.org/api/nltk.probability.html?highlight=freqdist#nltk.probability.FreqDist) verwenden.

In [4]:
# Lösung A1
from nltk.probability import FreqDist
from collections import defaultdict
# Brown corpus mit universal POS tags 
tagged_words = brown.tagged_words(tagset="universal")

In [5]:
# Durchschnittliche Anzahl der POS-Tags einer Wortform
word_to_tags = defaultdict(set)
for word, tag in tagged_words:
    word_to_tags[word.lower()].add(tag)

avg_tags_per_word_form = sum(len(tags) for tags in word_to_tags.values()) / len(word_to_tags)

print(f"1. Average number of POS tags per word form: {avg_tags_per_word_form:}")

1. Average number of POS tags per word form: 1.0749372678911975


In [6]:
# Die 10 Wortformen mit der höchsten Anzahl an POS-Tags
word_forms_sorted = sorted(word_to_tags.items(), key=lambda x: len(x[1]), reverse=True)
top_10_ambiguous_words = word_forms_sorted[:10]

print("2. Top 10 ambiguous word forms (word -> tags):")
for word, tags in top_10_ambiguous_words:
    print(f"   {word}: {tags}")

2. Top 10 ambiguous word forms (word -> tags):
   down: {'PRT', 'ADV', 'ADP', 'ADJ', 'NOUN', 'VERB'}
   that: {'ADV', 'ADP', 'DET', 'PRON', 'X'}
   to: {'PRT', 'ADV', 'ADP', 'NOUN', 'X'}
   well: {'ADV', 'PRT', 'ADJ', 'NOUN', 'VERB'}
   round: {'ADV', 'ADP', 'ADJ', 'NOUN', 'VERB'}
   damn: {'PRT', 'ADV', 'ADJ', 'NOUN', 'VERB'}
   in: {'NOUN', 'X', 'PRT', 'ADP'}
   many: {'ADJ', 'PRT', 'ADV', 'X'}
   best: {'NOUN', 'ADJ', 'ADV', 'VERB'}
   :: {'X', 'NOUN', 'ADP', '.'}


In [7]:
# %-Anteil der ambigen Wortformen bei den Wortformen
num_ambiguous_forms = sum(1 for tags in word_to_tags.values() if len(tags) > 1)
percent_ambiguous_forms = (num_ambiguous_forms / len(word_to_tags)) * 100

print(f"3. Percentage of ambiguous word forms: {percent_ambiguous_forms:}%")

3. Percentage of ambiguous word forms: 6.84131285757302%


In [8]:
# Durchschnittliche Anzahl der möglichen POS-Tags eines Worttokens
token_to_possible_tags = [len(word_to_tags[word.lower()]) for word, _ in tagged_words]
avg_tags_per_word_token = sum(token_to_possible_tags) / len(tagged_words)

print(f"4. Average number of possible POS tags per word token: {avg_tags_per_word_token:}")

4. Average number of possible POS tags per word token: 1.8337096707521237


In [9]:
# %-Anteil der Worttoken mit ambiger Wortform
num_ambiguous_tokens = sum(1 for tags in token_to_possible_tags if tags > 1)
percent_ambiguous_tokens = (num_ambiguous_tokens / len(tagged_words)) * 100

print(f"5. Percentage of word tokens with ambiguous word forms: {percent_ambiguous_tokens:}%")

5. Percentage of word tokens with ambiguous word forms: 55.67838910361077%


# Methoden des POS-Taggings

Im Folgenden werden wir einfache Methoden des POS-Taggings kennenlernen. Damit die jeweilige Leistungsfähigkeit gemessen und verglichen werden kann, müssen wir uns aber erst einmal mit dem Thema Evaluierung auseinander setzen.

## Evaluierung mit Trainings- und Testdaten

Für die Evaluierung der Tagger werden üblicherweise *Trainingsdaten* und *Testdaten* verwendet: 

- Die **Trainingsdaten** enthalten Beispiele für die konkrete Verwendung der POS-Tags. Bei überwachten Verfahren ist das die Grundlage, auf der der Tagger Zuweisung von POS-Tags lernen soll; 
- die **Testdaten** stammen meist aus demselben (hand-)annotierten Corpus, aber hier muss der Tagger die fehlenden POS-Tags einsetzen und das Ergebnis wird mit den "Goldtags" verglichen. 
- **Wichtig ist, dass sich Trainings- und Testdaten nicht überschneiden!**

Mit Hilfe des Brown Corpus lassen sich beide Datentypen leicht erstellen, indem zum Beispiel der ersten 400 Sätze zum Testen und die restlichen ca. 4000 Sätze zum Training verwendet werden:

In [10]:
from nltk.tag import untag

brown_testgold = brown.tagged_sents(categories='news', tagset='universal')[:400]
brown_train = brown.tagged_sents(categories='news', tagset='universal')[400:]
brown_test = [untag(sent) for sent in brown_testgold]

print("Testdaten:\n{}\n".format(brown_test[:2]))
print("Trainingsdaten:\n{}".format(brown_train[:2]))

Testdaten:
[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.']]

Trainingsdaten:
[[('He', 'PRON'), ('is', 'VERB'), ('not', 'ADV'), ('interested', 'VERB'), ('in', 'ADP'), ('being', 'VERB'), ('named', 'VERB'), ('a', 'DET'), ('full-time', 'ADJ'), ('director', 'NOUN'), ('.', '.')], [('Noting', 'VERB'), ('that', 'ADP'), ('President', 'NOUN'), ('Kennedy', 'NOUN'), ('has', 'VERB'), ('handed', 'VERB'), ('the', 'DET'), ('Defense', 'NOUN'), ('Departme

**ACHTUNG:** Die POS-Tagger werden immer auf einzelne tokenisierte Sätze angewandt. Die Satz- und Worttokenisierung muss also schon durchgeführt sein.

Man kann nun zwei Dinge anhand der Testdaten und der Goldtestdaten messen:
- Wie gut werden alle Worte klassifiziert?  $\Rightarrow$ **Accuracy** (global precision)
- Wie gut funktioniert die Klassifikation bei einzelnen POS-Tags? $\Rightarrow$ **Precision**, **Recall**, **F1 Measure**

\begin{align} 
R =&~ \frac{\#\text{correctly assigned POS tags}}{\#\text{POS tags in gold data}} \\
P =&~ \frac{\#\text{correctly assigned POS tags}}{\#\text{POS tags assigned}} \\
F1 =&~ \frac{2RP}{R+P}
\end{align}

Man sieht auch manchmal, dass Precision, Recall und F1-Measure auf das ganze Testset angewandt werden. Das ist beim POS-Tagging aber äquivalent zur Accuracy, da jedes Wort genau ein POS-Tag erhalten muss. Es gilt also in diesem Fall:

\begin{align}
\#\text{POS tags assigned} =&~ \#\text{POS tags in gold data} 
\end{align}

Und damit gilt auch $R = P$, so dass wir F1 folgendermaßen auflösen können:

\begin{align}
F1 = \frac{2RP}{R+P} = \frac{2P^2}{2P} = P
\end{align}

## Default-Tagger

Der Default-Tagger weist immer dasselbe POS-Tag zu, egal welches Wort vorliegt. Im Unterschied zu einem Random-Tagger, der aus den POS-Tags zufällig eines auswählt, kann der Default-Tagger aus den Testdaten immerhin "lernen", welches POS-Tag am häufigsten ist.

NLTK enthält bereits eine Klasse `DefaultTagger`, mit der in wenigen Zeilen ein Default-Tagger erstellt werden kann.

In [11]:
from nltk import DefaultTagger
dt = DefaultTagger('POS')
dt.tag(brown_test[0])

[('The', 'POS'),
 ('Fulton', 'POS'),
 ('County', 'POS'),
 ('Grand', 'POS'),
 ('Jury', 'POS'),
 ('said', 'POS'),
 ('Friday', 'POS'),
 ('an', 'POS'),
 ('investigation', 'POS'),
 ('of', 'POS'),
 ("Atlanta's", 'POS'),
 ('recent', 'POS'),
 ('primary', 'POS'),
 ('election', 'POS'),
 ('produced', 'POS'),
 ('``', 'POS'),
 ('no', 'POS'),
 ('evidence', 'POS'),
 ("''", 'POS'),
 ('that', 'POS'),
 ('any', 'POS'),
 ('irregularities', 'POS'),
 ('took', 'POS'),
 ('place', 'POS'),
 ('.', 'POS')]

Nun muss nur noch das häufigste POS-Tag in den Trainingsdaten ermittelt werden:

In [12]:
from nltk.probability import FreqDist

# Default-Tagger beim Lernen
mfpos = FreqDist([tag for sent in brown_train for word,tag in sent]).most_common(1)[0][0]

dt = DefaultTagger(mfpos)
dt.tag(brown_test[0])

[('The', 'NOUN'),
 ('Fulton', 'NOUN'),
 ('County', 'NOUN'),
 ('Grand', 'NOUN'),
 ('Jury', 'NOUN'),
 ('said', 'NOUN'),
 ('Friday', 'NOUN'),
 ('an', 'NOUN'),
 ('investigation', 'NOUN'),
 ('of', 'NOUN'),
 ("Atlanta's", 'NOUN'),
 ('recent', 'NOUN'),
 ('primary', 'NOUN'),
 ('election', 'NOUN'),
 ('produced', 'NOUN'),
 ('``', 'NOUN'),
 ('no', 'NOUN'),
 ('evidence', 'NOUN'),
 ("''", 'NOUN'),
 ('that', 'NOUN'),
 ('any', 'NOUN'),
 ('irregularities', 'NOUN'),
 ('took', 'NOUN'),
 ('place', 'NOUN'),
 ('.', 'NOUN')]

Für die Evaluation steht bei Tagger-Objekten glücklicherweise die Methode [`accuracy(gold)`](https://www.nltk.org/api/nltk.tag.api.html?highlight=evaluate#nltk.tag.api.TaggerI.accuracy) zur Verfügung. Bitte daran denken, dass jetzt die Goldtestdaten verwendet werden müssen:

In [13]:
dt.accuracy(brown_testgold)

0.30919679156136687

Eine Accuracy von 31,8% ist wirklich sehr niedrig – vor allem da wir wissen, dass nur ein POS-Tag verwendet wird. Der Informationsgewinn ist also gleich null.

## Regexp-Tagger

Während der Default-Tagger die Wortform ignoriert, geht es beim Regexp-Tagger darum, allein aus der Wortform das POS-Tag zu erschließen. Trainingsdaten werden also nicht benötigt. 

NLTK enthält hierfür die Klasse `RegexpTagger`, die wie `DefaultTagger` bedient werden kann (beide implementieren das Interface `TaggerI`):

In [14]:
from nltk import RegexpTagger
rt = RegexpTagger([
    (r'(The|the|A|a|An|an)$', 'DET'),      # articles
    (r'.*able$', 'ADJ'),                   # adjectives
    (r'.*ness$', 'NOUN'),                  # nouns formed from adjectives
    (r'.*ly$', 'ADV'),                     # adverbs
    (r'.*ing$', 'VERB'),                   # gerunds
    (r'.*ed$', 'VERB'),                    # simple past
    (r'.*es$', 'VERB'),                    # 3rd singular present
    (r'.*ould$', 'VERB'),                  # modals
    (r'.*\'s$', 'NOUN'),                   # possessive nouns
    (r'.*s$', 'NOUN'),                     # plural nouns
    (r'^-?[0-9]+(\.[0-9]+)?$', 'NUM'),     # cardinal numbers
    (r'(\.|\,|\(|\)|\;|\`\`|\'\')$', '.'), # punctuation
    (r'.*', 'NOUN'),                       # nouns (default)
    ])

rt.tag(brown_test[0])

[('The', 'DET'),
 ('Fulton', 'NOUN'),
 ('County', 'NOUN'),
 ('Grand', 'NOUN'),
 ('Jury', 'NOUN'),
 ('said', 'NOUN'),
 ('Friday', 'NOUN'),
 ('an', 'DET'),
 ('investigation', 'NOUN'),
 ('of', 'NOUN'),
 ("Atlanta's", 'NOUN'),
 ('recent', 'NOUN'),
 ('primary', 'NOUN'),
 ('election', 'NOUN'),
 ('produced', 'VERB'),
 ('``', '.'),
 ('no', 'NOUN'),
 ('evidence', 'NOUN'),
 ("''", '.'),
 ('that', 'NOUN'),
 ('any', 'NOUN'),
 ('irregularities', 'VERB'),
 ('took', 'NOUN'),
 ('place', 'NOUN'),
 ('.', '.')]

In [15]:
rt.accuracy(brown_testgold)

0.566311394352269

Die Accuracy ist nun deutlich erhöht, aber immer noch zu niedrig. Das grundsätzliche Problem hier ist, dass die regulären Ausdrücke nicht immer eindeutig einem POS-Tag zugeordnet werden können. Außerdem vergisst man leicht eine Regel, wodurch sich die Abdeckung verringert.

Um eine bessere Ahnung davon zu bekommen, welche POS-Tags korrekt und welche fälschlicherweise vergeben wurden, lohnt sich ein Blick auf die **Konfusionsmatrix**. NLTK stell hierfür das [`ConfusionMatrix`-Modul](https://www.nltk.org/api/nltk.metrics.confusionmatrix.html?highlight=confusion%20matrix#module-nltk.metrics.confusionmatrix) zur Verfügung.

In [16]:
from nltk.metrics import ConfusionMatrix

testtags = [tag for sent in brown_test for word,tag in rt.tag(sent)]
goldtags = [tag for sent in brown_testgold for word,tag in sent]

cm = ConfusionMatrix(goldtags, testtags)

print(cm.pretty_format(show_percents=True, values_in_chart=True, truncate=15, sort_by_count=True))

     |      N      V                                                C             P        |
     |      O      E      A      D             A      A      P      O      N      R        |
     |      U      R      D      E             D      D      R      N      U      O        |
     |      N      B      P      T      .      J      V      T      J      M      N      X |
-----+-------------------------------------------------------------------------------------+
NOUN | <28.7%>  2.1%      .   0.0%      .   0.0%   0.1%      .      .      .      .      . |
VERB |   9.1%  <6.5%>     .      .      .   0.0%   0.0%      .      .      .      .      . |
 ADP |  12.7%   0.1%     <.>     .      .      .      .      .      .      .      .      . |
 DET |   2.5%      .      .  <9.0%>     .      .      .      .      .      .      .      . |
   . |   0.4%      .      .      . <10.4%>     .      .      .      .      .      .      . |
 ADJ |   6.4%   0.1%      .      .      .  <0.1%>  0.1%      .      . 

Man erkennt, dass noch immer viel zu häufig das POS-Tag `NOUN` vergeben wird (siehe Spalte "NOUN"), das hier also die Regeln noch verfeinert werden müssen ...

## Lookup-Tagger oder Unigram-Tagger

Die Idee beim Lookup-Tagger ist, für jedes Wort das POS-Tag in einem "Wörterbuch" nachzuschlagen. Dieses Wörterbuch kann ad hoc aus den Trainingsdaten generiert werden, indem für jede Wortform das häufigste POS-Tag gespeichert wird.

Da man hier nur einzelne Wortformen und Token betrachtet, spricht man auch von einem **Unigram-Tagger**. Wie schon beim Default-Tagger hält NLTK eine passende Klasse ([`UnigramTagger`](https://www.nltk.org/api/nltk.tag.sequential.html?highlight=unigramtagger#nltk.tag.sequential.UnigramTagger)) bereit:

In [17]:
from nltk import UnigramTagger

# Unigram-Tagger beim Lernen
ut = UnigramTagger(brown_train)

ut.tag(brown_test[0])

[('The', 'DET'),
 ('Fulton', None),
 ('County', 'NOUN'),
 ('Grand', 'ADJ'),
 ('Jury', 'NOUN'),
 ('said', 'VERB'),
 ('Friday', 'NOUN'),
 ('an', 'DET'),
 ('investigation', 'NOUN'),
 ('of', 'ADP'),
 ("Atlanta's", 'NOUN'),
 ('recent', 'ADJ'),
 ('primary', 'NOUN'),
 ('election', 'NOUN'),
 ('produced', 'VERB'),
 ('``', '.'),
 ('no', 'DET'),
 ('evidence', 'NOUN'),
 ("''", '.'),
 ('that', 'ADP'),
 ('any', 'DET'),
 ('irregularities', None),
 ('took', 'VERB'),
 ('place', 'NOUN'),
 ('.', '.')]

Wieder können wir mit `accuracy()` die Accuracy des Taggers anhand von Goldtestdaten ermitteln:

In [18]:
ut.accuracy(brown_testgold)

0.8745192835952094

Das Ergebnis ist mit 88,1 % deutlich besser als beim Default-Tagger und beim Regexp-Tagger. 

Kann man sich damit zufrieden geben? 

Leider nein! Da die Accuracy bezogen auf Worte berechnet wird, bedeutet das, dass durchschnittlich etwa jedes 10. Wort falsch getaggt wird. In jedem durchschnittlich großen Satz von 20 Worten gibt es also durchschnittlich zwei Fehler. Diese Fehlerrate ist für eine Methode, die immer noch relativ am Anfang der Verarbeitungspipeline steht, viel zu hoch. 

## Backoff: The power of many

Wir müssen also weiterhin nach Verbesserungsmöglichkeiten Ausschau halten – und in der Ausgabe von `ut` fällt sofort eine Verbesserungsmöglichkeit auf: Manche Wortformen haben das POS-Tag `none` erhalten, weil sie in den Trainingsdaten nicht gesehen wurden. Das sind die sogenannten **unbekannten Worte** ("unknown words").

In [19]:
from nltk.metrics import ConfusionMatrix

testtags = [str(tag) for sent in brown_test for word,tag in ut.tag(sent)]
goldtags = [tag for sent in brown_testgold for word,tag in sent]

cm = ConfusionMatrix(goldtags, testtags)

print(cm.pretty_format(show_percents=True, values_in_chart=True, truncate=15, sort_by_count=True))

     |      N      V                                                C             P             N |
     |      O      E      A      D             A      A      P      O      N      R             o |
     |      U      R      D      E             D      D      R      N      U      O             n |
     |      N      B      P      T      .      J      V      T      J      M      N      X      e |
-----+--------------------------------------------------------------------------------------------+
NOUN | <24.4%>  0.5%      .   0.0%      .   0.2%   0.0%      .      .   0.0%   0.0%   0.0%   5.7% |
VERB |   0.7% <13.3%>  0.0%      .      .   0.0%   0.0%      .      .      .      .      .   1.6% |
 ADP |      .   0.0% <11.7%>     .      .      .   0.0%   1.0%      .      .      .      .   0.0% |
 DET |      .      .   0.0% <11.5%>     .      .      .      .      .      .      .      .   0.0% |
   . |      .      .      .      . <10.8%>     .      .      .      .      .      .      .      . |


Natürlich könnte man einfach das Lookup-Wörterbuch um weitere Wortformen erweitern, indem man das Lernkorpus vergrößert. Aber auch dieses Vorgehen hat Grenzen, wie das folgende Diagramm aus dem NLTK-Buch zeigt, denn es müssen immer mehr Daten (X-Achse) für immer weniger Performance-Verbesserung (Y-Achse) eingesetzt werden: 

![](https://www.nltk.org/images/tag-lookup.png)

Vielleicht gibt es eine bessere Strategie?

Eine ganz simple Strategie besteht darin, den Regexp-Tagger dann zu verwenden, wenn der Unigram-Tagger `none` ausgeben würde. Diese Strategie eines "geordneten Rückzugs" nennt man **Backoff**. 

Praktischerweise erlaubt NLTK bei der Tagger-Konstruktion, einen Backoff per Parameter `backoff=` anzugeben:

In [20]:
from nltk import UnigramTagger

# Unigram-Tagger beim Lernen
utplus = UnigramTagger(brown_train, backoff=rt)

Die Accuracy kann dadurch deutlich zulegen:

In [21]:
utplus.accuracy(brown_testgold)

0.9429732996374025

Was sich natürlich auch in der Konfusionsmatrix wiederspiegelt:

In [22]:
testtags = [str(tag) for sent in brown_test for word,tag in utplus.tag(sent)]
goldtags = [tag for sent in brown_testgold for word,tag in sent]

cm = ConfusionMatrix(goldtags, testtags)

print(cm.pretty_format(show_percents=True, values_in_chart=True, truncate=15, sort_by_count=True))

     |      N      V                                                C             P        |
     |      O      E      A      D             A      A      P      O      N      R        |
     |      U      R      D      E             D      D      R      N      U      O        |
     |      N      B      P      T      .      J      V      T      J      M      N      X |
-----+-------------------------------------------------------------------------------------+
NOUN | <29.9%>  0.8%      .   0.0%      .   0.2%   0.0%      .      .   0.0%   0.0%   0.0% |
VERB |   1.2% <14.3%>  0.0%      .      .   0.0%   0.0%      .      .      .      .      . |
 ADP |      .   0.0% <11.7%>     .      .      .   0.0%   1.0%      .      .      .      . |
 DET |   0.0%      .   0.0% <11.5%>     .      .      .      .      .      .      .      . |
   . |      .      .      .      . <10.8%>     .      .      .      .      .      .      . |
 ADJ |   1.2%   0.1%      .      .      .  <5.4%>  0.1%      .      . 

## N-Gram-Tagger: Mehr Kontext bitte!

Eine grundlegende Einschränkung des Lookup-Taggers (bzw. des Unigram-Taggers) ist, dass der Kontext des Worttokens nicht berücksichtigt wird. Der Kontext ist aber oft wichtig, da vielen Wortformen mehr als ein POS-Tag zugewiesen werden kann.

In [23]:
ut.tag(nltk.word_tokenize("A fly flies to flies in order to fly."))

[('A', 'DET'),
 ('fly', 'NOUN'),
 ('flies', 'VERB'),
 ('to', 'PRT'),
 ('flies', 'VERB'),
 ('in', 'ADP'),
 ('order', 'NOUN'),
 ('to', 'PRT'),
 ('fly', 'NOUN'),
 ('.', '.')]

Der Ansatz des N-Gram-Taggers ist daher, den unmittelbaren, vorangehenden Kontext bei der Klassifizierung einzubeziehen. "N" steht für die festgelegte Größe des Kontexts: Ein **2-Gram-Tagger** (oder Bigram-Tagger) betrachtet z.B. das zu klassifizierende Wort und das POS-Tag des davorstehenden Worts. Der N-Gram-Tagger ist also eine Generalisierung des Unigram-Taggers. Schematisch kann das folgendermaßen dargestellt werden (Darstellung aus https://www.nltk.org/book/ch05.html), wobei sich die Bigramme in dieser Darstellung nur über $t_{n-1}$ und $w_n$ erstrecken: 

![](https://www.nltk.org/images/tag-context.png)

Für Bigram-Tagger bedeutet das, dass während der Lernphase für alle Token der Lerndaten Tripel der Form 

     (POS-Tag des vorangehenden Tokens, Wortform des Tokens, POS-Tag des Tokens) 
   
erstellt werden. Beim Taggen eines Satzes $(w_1,t_1) \ldots (w_k,t_k)$ wird dann für ein Token mit Wortform $w_i$ das POS-Tag $t_i$ gewählt, falls $(t_{i-1},w_i,t_i)$ für $t_{i-1}$ und $w_i$ am häufigsten in den Lerndaten vorkam. 

N-Grams von Listen können mit Hilfe des NLTK-Moduls [`ngrams`](https://www.nltk.org/api/nltk.util.html?highlight=ngram#nltk.util.ngrams) leicht erstellt werden: 

In [24]:
from nltk import ngrams
list(ngrams([1,2,3,4],2))

[(1, 2), (2, 3), (3, 4)]

Die Tupel einens N-Gram-Taggers sehen also konkret folgendermaßen aus:

In [25]:
n = 2 
input = brown_train[0]
print(input)
[[tag for word,tag in ngram[:-1]]+[ngram[-1][0]]+[ngram[-1][1]] for ngram in ngrams(input, n)]

[('He', 'PRON'), ('is', 'VERB'), ('not', 'ADV'), ('interested', 'VERB'), ('in', 'ADP'), ('being', 'VERB'), ('named', 'VERB'), ('a', 'DET'), ('full-time', 'ADJ'), ('director', 'NOUN'), ('.', '.')]


[['PRON', 'is', 'VERB'],
 ['VERB', 'not', 'ADV'],
 ['ADV', 'interested', 'VERB'],
 ['VERB', 'in', 'ADP'],
 ['ADP', 'being', 'VERB'],
 ['VERB', 'named', 'VERB'],
 ['VERB', 'a', 'DET'],
 ['DET', 'full-time', 'ADJ'],
 ['ADJ', 'director', 'NOUN'],
 ['NOUN', '.', '.']]

Glücklicherweise gibt es in NLTK bereits eine passende Klasse [`NgramTagger`](https://www.nltk.org/api/nltk.tag.sequential.html?highlight=ngramtagger#nltk.tag.sequential.NgramTagger), mit der wir N-Gram-Tagger mit beliebigem N erzeugen können.

In [26]:
from nltk import NgramTagger

# der Bigram-Tagger trainiert ...
bigt = NgramTagger(2, brown_train)

Wenn wir damit den Satz von oben taggen, werden wir allerdings feststellen, dass ab dem zweiten *flies* alle Worte das POS-Tag `None` erhalten.  

In [27]:
bigt.tag(nltk.word_tokenize("A fly flies to flies in order to fly."))

[('A', 'DET'),
 ('fly', 'NOUN'),
 ('flies', 'VERB'),
 ('to', 'PRT'),
 ('flies', None),
 ('in', None),
 ('order', None),
 ('to', None),
 ('fly', None),
 ('.', None)]

Das liegt daran, dass in den Trainingsdaten *flies* nicht hinter `PRT` auftritt und deswegen als `None` klassifiziert wird. D.h. es gibt in den Lerndaten kein Tupel wie `['PRT','flies','VERB']`. Da aber `None` für den Bigram-Tagger ebenfalls ein unbekannter Kontext ist, setzt sich der Fehler bis zum Ende fort.

Um dem entgegenzuwirken, muss man eine Rückfall-Strategie ("backoff") angeben. Das ist mit der Option `backoff` möglich und hier setzen wir einfach den Unigram-Tagger von oben ein:

In [28]:
from nltk import NgramTagger

bigt = NgramTagger(2, brown_train,backoff=ut)
bigt.tag(nltk.word_tokenize("A fly flies to flies in order to fly."))

[('A', 'DET'),
 ('fly', 'NOUN'),
 ('flies', 'VERB'),
 ('to', 'PRT'),
 ('flies', 'VERB'),
 ('in', 'ADP'),
 ('order', 'NOUN'),
 ('to', 'PRT'),
 ('fly', 'VERB'),
 ('.', '.')]

In der Evaluation sehen wir, dass der Bigram-Tagger mit dem Unigram-Tagger als Backoff zu leicht verbesserter Accuracy führt:

In [29]:
bigt.accuracy(brown_testgold)

0.8810020876826722

Nun bedeutet "mehr" nicht notwendigerweise "besser". Wenn wir nämlich stattdessen einen **Trigram-Tagger** implementieren und damit mehr Kontext berücksichtigen, dann erhalten wir im Vergleich zum Bigram-Tagger eine leicht *verringerte* Accuracy: 

In [30]:
trigt = NgramTagger(3, brown_train, backoff=ut)
trigt.accuracy(brown_testgold)

0.8757279419843973

Der Grund dafür ist die unvermeidbare Datenknappheit: Der Trigram-Tagger wird häufiger als der Bigram-Tagger in die Situation kommen, dass ein Trigram in den Lerndaten nicht gesehen wurde und daher das Ergebnis des Backoff-Taggers übernommen werden muss. Die Accuracy der N-Gram-Tagger nähert sich also mit wachsendem N der Accuracy des Backoffs an.

## <span style="color:red">Aufgaben II</span>

Wir sind nun bereit für den letzten Schritt.

<span style="color:red">A2:</span> Implementieren Sie einen POS-Tagger `ngt`, der die Tagger per `backoff` so kombiniert, dass die Accuracy besser als bei `utplus` oben ausfällt! 

In [31]:
# Lösung A2

# Unigram-Tagger 
unigram_tagger = UnigramTagger(brown_train, backoff=rt)

# Ngram-Tagger: Bigram und Trigram taggers mit backoff Kombinieren
bigram_tagger = NgramTagger(2, brown_train, backoff=unigram_tagger)
trigram_tagger = NgramTagger(3, brown_train, backoff=bigram_tagger)

# Final POS Tagger (ngt)
ngt = trigram_tagger

In [32]:
# Konfusionsmatrix

testtags = [str(tag) for sent in brown_test for word,tag in ngt.tag(sent)]
goldtags = [tag for sent in brown_testgold for word,tag in sent]

cm = ConfusionMatrix(goldtags, testtags)

print(cm.pretty_format(show_percents=True, values_in_chart=True, truncate=15, sort_by_count=True))
print("Accuracy: {}".format(ngt.accuracy(brown_testgold)))

     |      N      V                                                C             P        |
     |      O      E      A      D             A      A      P      O      N      R        |
     |      U      R      D      E             D      D      R      N      U      O        |
     |      N      B      P      T      .      J      V      T      J      M      N      X |
-----+-------------------------------------------------------------------------------------+
NOUN | <30.1%>  0.6%      .      .      .   0.2%      .      .      .   0.0%   0.0%      . |
VERB |   1.0% <14.5%>  0.0%      .      .   0.0%   0.0%      .      .      .      .      . |
 ADP |      .   0.0% <12.0%>  0.0%      .      .   0.1%   0.6%      .      .   0.1%      . |
 DET |   0.0%      .      . <11.5%>     .      .   0.0%      .      .      .      .      . |
   . |      .      .      .      . <10.8%>     .      .      .      .      .      .      . |
 ADJ |   1.3%   0.1%      .      .      .  <5.3%>  0.1%      .      . 

## Achtung Domänenabhängigkeit!

Bei aller Euphorie: Die Performance-Ergebnisse gelten nur für eine bestimmte **Domäne**, d.h. eine bestimmte Sprache und Textsorte. 

Angewandt auf eine andere Domäne als "news" wird die Accuracy *wahrscheinlich* dahinschmelzen. Probieren wir es aus:

In [33]:
print("{:20} {:10}".format("Domäne","Accuracy"))
print("{:20} {:10}".format("------","--------"))

for category in brown.categories():
    print("{:20} {:10}".format(category, ngt.accuracy(
    brown.tagged_sents(categories=category, tagset='universal'))))

Domäne               Accuracy  
------               --------  
adventure            0.919442761962447
belles_lettres       0.9181205804871285
editorial            0.9242743977663788
fiction              0.9202634038079663
government           0.9282770226906456
hobbies              0.9095148460744429
humor                0.9172620419451486
learned              0.914843200211119
lore                 0.9217581301734377
mystery              0.9229827353985551
news                 0.9794140461841399
religion             0.9194141983299069
reviews              0.9120970911949685
romance              0.9181400131387278
science_fiction      0.9192812715964064


# Weitere Arten von Tagger

Neben den N-Gram-Taggern gibt es noch eine Reihe weiterer stochastischer Tagger in NLTK, die wesentlich elaborierter sind: 
- [Hidden Markov Models](https://www.nltk.org/api/nltk.tag.hmm.html?highlight=markov#nltk.tag.hmm.HiddenMarkovModelTagger) ($\to$ nächste Sitzung)
- [Conditional Random Fields](https://www.nltk.org/api/nltk.tag.crf.html?highlight=crf#module-nltk.tag.crf) ($\to$ nächste Sitzung)
- [Perceptron](https://www.nltk.org/api/nltk.tag.perceptron.html?highlight=tagger#module-nltk.tag.perceptron)
- ["klassifikationsbasiert"](https://www.nltk.org/api/nltk.tag.sequential.html?highlight=tagger#nltk.tag.sequential.ClassifierBasedPOSTagger)

Wir werden eine Auswahl dieser Tagger in der nächsten Sitzung noch behandeln. 

## Brill-Tagger

Zum Abschluss möchte ich auf einen besonderen Tagger eingehen, der ebenfalls statistische und regelbasierte Ansätze verbinden kann: der [**Brill-Tagger**](https://en.wikipedia.org/wiki/Brill_tagger), benannt nach Eric Brill und entstanden Anfang der 90er ([Brill 1992](https://aclanthology.org/H92-1022)). 

Das besondere an diesem Tagger ist, dass er zwei Verarbeitungsschritte hat: 
1. Zunächst wird mit einem bestehenden **Baseline-Tagger** eine Eingabe verarbeitet. 
2. Dann wird die Ausgabe mithilfe von Korrekturregeln, den sogenannten **Patches**, verbessert.   

Die Patches werden automatisch aus den Fehlern des Baseline-Taggers in einem Testset induziert, die man ja anhand der Golddaten ermitteln kann. Ein Patch hat die Form einer bedingten Ersetzungsregel und kann z.B. folgendermaßen ausehen:

    NOUN->ADP if Word:of@[0]       ('Ersetze NOUN durch ADP, falls die Wortform /of/ ist.')

Der Bedingungsteil der Patches kann prinzipiell beliebige Kontextinformationen und deren Kombination enthalten. Um den Suchraum bei der Induktion einzugrenzen, muss daher zuvor die zur Verfügung stehende Kontextinformation durch **Patch Templates** festgelegt werden. Dies erfolgt manuell, wobei der Aufwand und die Fehleranfälligkeit (es gibt keine schädlichen Patch Templates) sehr gering ist. 

NLTK beinhaltet z.B. die 24 Templates des ursprünglichen Brill-Tagger:   

In [34]:
nltk.tag.brill.brill24()

[Template(Pos([-1])),
 Template(Pos([1])),
 Template(Pos([-2])),
 Template(Pos([2])),
 Template(Pos([-2, -1])),
 Template(Pos([1, 2])),
 Template(Pos([-3, -2, -1])),
 Template(Pos([1, 2, 3])),
 Template(Pos([-1]),Pos([1])),
 Template(Pos([-2]),Pos([-1])),
 Template(Pos([1]),Pos([2])),
 Template(Word([-1])),
 Template(Word([1])),
 Template(Word([-2])),
 Template(Word([2])),
 Template(Word([-2, -1])),
 Template(Word([1, 2])),
 Template(Word([-1, 0])),
 Template(Word([0, 1])),
 Template(Word([0])),
 Template(Word([-1]),Pos([-1])),
 Template(Word([1]),Pos([1])),
 Template(Word([0]),Word([-1]),Pos([-1])),
 Template(Word([0]),Word([1]),Pos([1]))]

Liegen die Templates in dieser Form vor, kann der Brill-Tagger mithilfe der Ausgabe eines Baseline-Taggers die Patches induzieren. Hierfür steht in NLTK die Klasse [`BrillTaggerTrainer`](https://www.nltk.org/api/nltk.tag.brill_trainer.html?highlight=taggertrainer#nltk.tag.brill_trainer.BrillTaggerTrainer) zur Verfügung. Mittels der Funktion [`train(train_sents, max_rules=200, min_score=2, min_acc=None)`](https://www.nltk.org/api/nltk.tag.brill_trainer.html?highlight=taggertrainer#nltk.tag.brill_trainer.BrillTaggerTrainer.train) wird der Trainingsvorgang angestoßen.

In [35]:
baseline = rt
learndata = brown_train
templates = nltk.tag.brill.brill24()

btt = nltk.tag.brill_trainer.BrillTaggerTrainer(baseline, templates, trace=3)
brilltagger = btt.train(learndata, max_rules=200)

TBL train (fast) (seqs: 4223; tokens: 91453; tpls: 24; min score: 2; min acc: None)
Finding initial useful rules...
    Found 223614 useful rules.

           B      |
   S   F   r   O  |        Score = Fixed - Broken
   c   i   o   t  |  R     Fixed = num tags changed incorrect -> correct
   o   x   k   h  |  u     Broken = num tags changed correct -> incorrect
   r   e   e   e  |  l     Other = num tags changed incorrect -> incorrect
   e   d   n   r  |  e
------------------+-------------------------------------------------------
38624007 1451848  | NOUN->ADP if Pos:DET@[1]
18001800   0   0  | NOUN->CONJ if Word:and@[0]
16701670   0   0  | NOUN->ADP if Word:of@[0]
10861087   1 471  | NOUN->PRT if Word:to@[0]
10661066   0  40  | NOUN->ADP if Word:in@[0]
 635 643   8  38  | NOUN->VERB if Word:be@[0,1]
 618 638  20 135  | NOUN->VERB if Word:was@[-1,0]
 543 543   0   1  | NOUN->ADP if Word:for@[0]
 541 541   0   0  | NOUN->VERB if Word:is@[0]
 456 696 240 159  | NOUN->VERB if Pos:PRT@[-1

In [36]:
print("Accuracy des Baseline-Taggers: ",baseline.accuracy(brown_testgold))
print("Accuracy des Brill-Taggers: ",brilltagger.accuracy(brown_testgold))

Accuracy des Baseline-Taggers:  0.566311394352269
Accuracy des Brill-Taggers:  0.8668278211185584


Der Brill-Tagger hat im Allgemeinen drei Vorteile (siehe [Brill 1992](https://aclanthology.org/H92-1022)):
- Portabilität: Baseline-Tagger können leicht für andere Tagsets, Textgenres und Sprachen angepasst werden.
- Geringer Speicherverbrauch: Die Anzahl der Regeln ist sehr gering verglichen mit N-Gram-Taggern. 
- Transparenz: Die Patches sind leicht verständlich und erweiterbar. 

# Appendix: Konkordanzen

Konkordanzen sind alignierte Trefferlisten ([Wikipedia](https://de.wikipedia.org/wiki/Konkordanz_\(Textwissenschaft\)#Konkordanzsoftware)), wobei die Suchanfragen nicht nur Wortformen, sondern auch POS-Tags enthalten können. Konkordanzen sind in der Linguistik sehr wichtig für die Datenrecherche, denn sie stellen gefundene Belege übersichtlich dar. 

![](https://upload.wikimedia.org/wikipedia/de/d/d4/Konkordanz_Nationalrat.png)

NLTK enthält ebenfalls einen sogenannten **Concordancer**, wobei hierfür ein Corpus zunächst in ein [Text-Objekt](https://www.nltk.org/api/nltk.text.html?highlight=text#nltk.text.Text) überführt werden muss. Mittels der Funktion [`concordance(word, width=79, lines=25)`](https://www.nltk.org/api/nltk.text.html?highlight=text#nltk.text.Text.concordance) können dann Konkordanzen ausgelesen werden.  

In [37]:
from nltk.text import Text
text = Text(brown.words())
text.concordance("monstrous")

Displaying 13 of 13 matches:
 couple of weeks ago , he scored a monstrous 12 on a par-5 hole . It made him h
ework or his plates feel loose and monstrous . His bifocals blur . His legs sud
he Soviet Union stands guilty of a monstrous crime against the human race . But
sment of the joke gone wrong , the monstrous image of the fat man dressed up as
ement was for Copernicus literally monstrous : `` With ( the Ptolemaists ) it i
ceptions were quick and his energy monstrous , but these qualities were sapped 
U.S. . It was `` the creation of a monstrous historical period wherein it thoug
me might be 38 , and occasional `` monstrous freaks '' over 50 . He rejects dim
t least 37 feet and the 50-foot `` monstrous freaks '' intimated by Heuvelmans 
st unnatural actions , of the most monstrous murders , told with the most spont
spicion darted into his mind . Too monstrous , of course . Mae wouldn't have pl
 . The sound was coming nearer . A monstrous shadow fell across the illuminated
And once be

Darüber hinaus steht eine GUI zur Verfügung, bei dem auch die Wortarten angezeigt und in die Suche einbezogen werden können.

In [38]:
#nltk.download('universal_tagset')
#nltk.app.concordance()

# Literaturangaben

- Brill, Eric. 1992. A simple rule-based part of speech tagger. In Speech and Natural Language: Proceedings of a Workshop Held at Harriman, New York, February 23–26, 1992, 112–116. Association for Computational Linguistics. https://aclanthology.org/H92-1022.